In [ ]:
# !kill -9 1249830

In [12]:
!nvidia-smi

Wed Jun  7 12:27:11 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000                On | 00000000:23:00.0 Off |                  Off |
| 30%   37C    P8               20W / 300W|   7331MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [13]:
import numpy as np
import argparse
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100, CIFAR10

import clip

In [49]:
model_name = 'RN50x64'
batch_size = 100
epochs = 10

available_model = clip.available_models()

print(available_model)

assert model_name in available_model

['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']


In [52]:
model.visual

ModifiedResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU(inplace=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU(inplace=True)
  (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 

In [51]:
model, preprocess = clip.load(model_name)
model.cuda().eval()

input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)
print(preprocess)

Model parameters: 623,258,305
Input resolution: 448
Context length: 77
Vocab size: 49408
Compose(
    Resize(size=448, interpolation=bicubic, max_size=None, antialias=None)
    CenterCrop(size=(448, 448))
    <function _convert_image_to_rgb at 0x7f42d7957ee0>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)


In [33]:
num_classes = len(cifar10_train.classes)
print(num_classes)

linear_probe_model = nn.Sequential(
    model.visual,
    nn.Linear(1024, num_classes, dtype=torch.float16)
)
linear_probe_model.eval()

10


Sequential(
  (0): ModifiedResNet(
    (conv1): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (conv2): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(40, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu3): ReLU(inplace=True)
    (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(80, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
    

In [19]:
device = "cuda"

linear_probe_model = linear_probe_model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(linear_probe_model.parameters(), lr=1e-4, momentum=0.9)

In [23]:
set_requires_grad_for_layer(linear_probe_model[0], False)
set_requires_grad_for_layer(linear_probe_model[1], True)

In [22]:

# Accuracy
def get_correct_samples(scores: torch.Tensor, labels: torch.Tensor) -> int:
    """Gets the number of correctly classified examples.

    Args:
        scores: the scores predicted with the network.
        labels: the class labels.

    Returns: 
        the number of correct samples.
    """
    classes_predicted = torch.argmax(scores, 1)
    return (classes_predicted == labels).sum().item()


def set_requires_grad_for_layer(layer: torch.nn.Module, train: bool) -> None:
    """Sets the attribute requires_grad to True or False for each parameter.
        
        Args:
            layer: the layer to freeze.
            train: if true train the layer.
    """
    for p in layer.parameters():
        p.requires_grad = train
     

In [24]:
for epoch in tqdm(range(epochs)):
    linear_probe_model.train()
    
    _loss= 0
    _correct = 0
    _total= 0
    
    for i, data in enumerate(cifar10_train_dataloader):
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        
        optimizer.zero_grad()
        
        outputs = linear_probe_model(inputs)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        _loss += loss.item()
        _total += labels.size(0)
        _correct += get_correct_samples(outputs, labels)

        
    print(f"----train----\nloss: {_loss / _total}    accuracy: {_correct / _total * 100}\n")
    
    model.eval()
    
    _loss= 0
    _correct = 0
    _total= 0
    
    best_acc = 0
    best_epoch = 0
    
    linear_probe_model.eval()
    
    for i, data in enumerate(cifar10_valid_dataloader):
        with torch.no_grad():
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()

            optimizer.zero_grad()

            outputs = linear_probe_model(inputs)

            loss = criterion(outputs, labels)

            _loss += loss.item()
            _total += labels.size(0)
            _correct += get_correct_samples(outputs, labels)
            
    print(f"----test----\nloss: {_loss / _total}    accuracy: {_correct / _total * 100}\n")
    
    if best_acc < (_correct / _total * 100):
        best_acc = _correct / _total * 100
        best_epoch = epoch
        
print(f"Best Accuracy: {best_acc} Best EPOCH: {best_epoch}")

  0%|                                                                                            | 0/10 [00:00<?, ?it/s]

----train----
loss: 0.0230120703125    accuracy: 10.85



 10%|████████▍                                                                           | 1/10 [01:28<13:14, 88.29s/it]

----test----
loss: 0.0230015625    accuracy: 11.12

----train----
loss: 0.02299171875    accuracy: 12.284



 20%|████████████████▊                                                                   | 2/10 [02:56<11:47, 88.41s/it]

...

----test----
loss: 0.022906640625    accuracy: 19.53

----train----
loss: 0.02290203125    accuracy: 19.99



 90%|███████████████████████████████████████████████████████████████████████████▌        | 9/10 [13:20<01:29, 89.12s/it]

----test----
loss: 0.022896875    accuracy: 20.04

----train----
loss: 0.0228920703125    accuracy: 20.772



100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [14:49<00:00, 88.94s/it]

----test----
loss: 0.022887890625    accuracy: 20.71

Best Accuracy: 20.71 Best EPOCH: 9


In [26]:
model

CLIP(
  (visual): ModifiedResNet(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu3): ReLU(inplace=True)
    (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
     